In [1]:
#INTEGRANTES:
# Camila Arbeláez - 1036959719
# Juan Pablo Botero - 98714459
# Gladis Carmona - 43983972

In [2]:
# Agrege acá el código para importar las librerias

# La libreria para "encontrar el sevicio" de Spark
import findspark
findspark.init()

# Librerias para "gestionar el servicio" de Spark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession

# Creamos una aplicación Spark en el Servicio
# Tenga cuidado con las tildes o caracteres especiales en el nombre de la app
AppSpark = SparkConf().setAppName("Evaluacion iSofware")

# definimos un espacio o contexto para la App
ContextoSpark=SparkContext.getOrCreate(conf=AppSpark)

# inicio una sesión en el espacio de la App
SesionSpark = SparkSession(ContextoSpark)

# inicio del espacio o contexto SQL
ContextoSql = SQLContext(sparkContext=ContextoSpark, sparkSession=SesionSpark)

In [3]:
!wget https://raw.githubusercontent.com/jpablo146unal/Trabajo_SparkQL/main/Asistencia_del_Pitch.csv
!wget https://raw.githubusercontent.com/jpablo146unal/Trabajo_SparkQL/main/Evaluacion_del_Pitch.csv

--2021-05-07 01:32:05--  https://raw.githubusercontent.com/jpablo146unal/Trabajo_SparkQL/main/Asistencia_del_Pitch.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93121 (91K) [text/plain]
Saving to: 'Asistencia_del_Pitch.csv'

Asistencia_del_Pitc 100%[===================>]  90.94K  --.-KB/s    in 0.02s   

2021-05-07 01:32:05 (4.08 MB/s) - 'Asistencia_del_Pitch.csv' saved [93121/93121]

--2021-05-07 01:32:06--  https://raw.githubusercontent.com/jpablo146unal/Trabajo_SparkQL/main/Evaluacion_del_Pitch.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting re

In [4]:
# 1. Cargue los datos en la carpeta datalake y luego del /datalake al HDFS (Hadoop File System)
# Recuerda usar ! para ejecutar el comando en el shell.
!hdfs dfs -copyFromLocal Asistencia_del_Pitch.csv /tmp/
!hdfs dfs -copyFromLocal Evaluacion_del_Pitch.csv /tmp/
!hdfs dfs -ls /tmp/*csv

-rw-r--r--   1 root supergroup      93121 2021-05-07 01:32 /tmp/Asistencia_del_Pitch.csv
-rw-r--r--   1 root supergroup     111944 2021-05-07 01:32 /tmp/Evaluacion_del_Pitch.csv


In [5]:
 # 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
asistencia =SesionSpark.read.load('/tmp/Asistencia_del_Pitch.csv',format="csv",sep=',',inferSchema='true',header='true')

In [9]:
 # 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
evaluacion =SesionSpark.read.load('/tmp/Evaluacion_del_Pitch.csv',format="csv",sep=',',inferSchema='true',header='true')

In [10]:
 # 2. Cree dos tablas SparkSQL y almacene el csv en su correspondiente tabla.
asistencia.select(['Marca temporal', 'Nombre de usuario', 'Equipo al que perteneces:', 'Equipo que va a exponer:']).show(5)

+--------------------+--------------------+-------------------------+------------------------+
|      Marca temporal|   Nombre de usuario|Equipo al que perteneces:|Equipo que va a exponer:|
+--------------------+--------------------+-------------------------+------------------------+
|2021/04/08 8:07:5...|jppinedal@unal.ed...|     Pertenezco a: Ser...|    Expone: Servicios...|
|2021/04/08 8:09:3...|dgarciabl@unal.ed...|     Pertenezco a: Lo ...|    Expone: Lo tengo ...|
|2021/04/08 8:09:5...|auarbelaeza@unal....|     Pertenezco a: Ges...|    Expone: Te lo cam...|
|2021/04/08 8:09:5...|jmunozhe@unal.edu.co|     Pertenezco a: Lo ...|    Expone: Te lo cam...|
|2021/04/08 8:10:0...|dgerenal@unal.edu.co|     Pertenezco a: Te ...|    Expone: Te lo cam...|
+--------------------+--------------------+-------------------------+------------------------+
only showing top 5 rows



In [11]:
#reemplazando espacios por underline
new_column_name_list = list(map(lambda x: x.replace(" ", "_"), asistencia.columns))
asistencia = asistencia.toDF(*new_column_name_list)
#reemplazando : por nada
new_column_name_list = list(map(lambda x: x.replace(":", ""), asistencia.columns))
asistencia = asistencia.toDF(*new_column_name_list)

In [12]:
asistencia.printSchema()

root
 |-- Marca_temporal: string (nullable = true)
 |-- Nombre_de_usuario: string (nullable = true)
 |-- Equipo_al_que_perteneces: string (nullable = true)
 |-- Equipo_que_va_a_exponer: string (nullable = true)



In [13]:
#reemplazando espacios por underline
new_column_name_list = list(map(lambda x: x.replace(" ", "_"), evaluacion.columns))
evaluacion = evaluacion.toDF(*new_column_name_list)
#reemplazando : por nada
new_column_name_list = list(map(lambda x: x.replace(":", ""), evaluacion.columns))
evaluacion = evaluacion.toDF(*new_column_name_list)

In [14]:
evaluacion.printSchema()

root
 |-- Marca_temporal: string (nullable = true)
 |-- Nombre_de_usuario: string (nullable = true)
 |-- Equipo_que_vas_a_evaluar: string (nullable = true)
 |-- Introducción_El_equipo_responde_adecuadamente_¿Quiénes_son_y_por_qué_están_aquí?: integer (nullable = true)
 |-- Equipo_El_equipo_responde_adecuadamente_¿Quiénes_están_detrás_de_la_idea_y_cuál_es_su_función?: integer (nullable = true)
 |-- Problema_El_equipo_responde_adecuadamente_¿Qué_problema_resolverá?,_¿es_realmente_un_problema?: integer (nullable = true)
 |-- Ventajas_El_equipo_responde_adecuadamente_¿Por_qué_su_solución_es_especial?,_¿qué_la_hace_distinta_de_otras?: integer (nullable = true)
 |-- Solución_El_equipo_responde_adecuadamente_¿Cómo_piensa_resolver_el_problema?: integer (nullable = true)
 |-- Producto_El_equipo_responde_adecuadamente_¿Cómo_funciona_el_producto_o_servicio?_Muestra_algunos_ejemplos.: integer (nullable = true)
 |-- Tracción_El_equipo_responde_adecuadamente_si_cuenta_con_clientes_que_demuestran_pot

In [15]:
!hadoop fs -put Asistencia_del_Pitch.csv
!hadoop fs -put Evaluacion_del_Pitch.csv
!hadoop fs -ls

Found 2 items
-rw-r--r--   1 root supergroup      93121 2021-05-07 01:33 Asistencia_del_Pitch.csv
-rw-r--r--   1 root supergroup     111944 2021-05-07 01:33 Evaluacion_del_Pitch.csv


In [16]:
###2. Creación tabla asistencia
###
SesionSpark.sql("""
CREATE TABLE 
    asistencia 
USING com.databricks.spark.csv 
OPTIONS (
    sep=',',
    path 'Asistencia_del_Pitch.csv', 
    header 'true')
""");

In [17]:
###2. Creación tabla evaluacion
SesionSpark.sql("""
CREATE TABLE 
    evaluacion 
USING com.databricks.spark.csv 
OPTIONS (
    sep=',',
    path 'Evaluacion_del_Pitch.csv', 
    header 'true')
""");

In [18]:
###Confirmación creación de tablas
SesionSpark.sql("SHOW TABLES").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default|asistencia|      false|
| default|evaluacion|      false|
+--------+----------+-----------+



In [19]:
# 2. Consulte el listado total de estudiantes (correos electrónicos) del 
# curso de Ingeniería de Software, ordenados alfabéticamente
asistencia.createOrReplaceTempView("asistencia1")
SesionSpark.sql("select distinct(Nombre_de_Usuario) from asistencia1 order by(Nombre_de_Usuario) ").show()

+--------------------+
|   Nombre_de_Usuario|
+--------------------+
| abanolc@unal.edu.co|
|acastrillonv@unal...|
| aflemag@unal.edu.co|
|anapariciom@unal....|
|angutierrezb@unal...|
|anoriega@unal.edu.co|
|auarbelaeza@unal....|
| bocampo@unal.edu.co|
|cgiraldo@unal.edu.co|
|cjfunezg@unal.edu.co|
|cquinchiar@unal.e...|
| dadazam@unal.edu.co|
|daestradam@unal.e...|
|davgarciava@unal....|
|dballesteroso@una...|
|  dbrito@unal.edu.co|
|dcadavid@unal.edu.co|
|dcardonaal@unal.e...|
|dchavarriar@unal....|
|dgarciabl@unal.ed...|
+--------------------+
only showing top 20 rows



In [20]:
# 3. Consulte la cantidad de asistencias registradas por estudiante; además, la fecha y hora de la primera asistencia
# y la fecha y hora de la última asistencia
# Tu código a continuación...
asistencia.createOrReplaceTempView("asistencia1")
SesionSpark.sql("select (Nombre_de_Usuario), count(Marca_Temporal), min(Marca_Temporal), max(Marca_Temporal) from asistencia1 group by(Nombre_de_Usuario) ").show()

+--------------------+---------------------+--------------------+--------------------+
|   Nombre_de_Usuario|count(Marca_Temporal)| min(Marca_Temporal)| max(Marca_Temporal)|
+--------------------+---------------------+--------------------+--------------------+
|samolinap@unal.ed...|                   10|2021/04/08 8:11:4...|2021/04/13 9:37:2...|
| jruedat@unal.edu.co|                    8|2021/04/08 8:12:5...|2021/04/13 9:41:2...|
|nvalenciat@unal.e...|                    8|2021/04/08 8:10:1...|2021/04/13 9:39:3...|
|serendona@unal.ed...|                    8|2021/04/08 8:10:4...|2021/04/13 9:40:2...|
|jgutierrezlo@unal...|                    8|2021/04/08 8:12:0...|2021/04/13 9:39:0...|
|dgerenal@unal.edu.co|                    8|2021/04/08 8:10:0...|2021/04/13 9:41:2...|
|cjfunezg@unal.edu.co|                    8|2021/04/08 8:12:1...|2021/04/13 9:41:4...|
|emflorezb@unal.ed...|                    8|2021/04/08 8:14:0...|2021/04/13 9:45:4...|
|dchavarriar@unal....|                    8

In [21]:
# 4. Consulte el listado de estudiantes que asistieron a 2 presentaciones o menos (una).
# Tu código a continuación...
asistencia.createOrReplaceTempView("asistencia1")
SesionSpark.sql("select (Nombre_de_Usuario), count(Marca_Temporal) from asistencia1 group by(Nombre_de_Usuario) having count(Marca_Temporal) in (1,2) ").show()

+--------------------+---------------------+
|   Nombre_de_Usuario|count(Marca_Temporal)|
+--------------------+---------------------+
|jdroldano@unal.ed...|                    1|
+--------------------+---------------------+



In [22]:
# 5. Consulte el listado de estudiantes que no asistieron a ninguna presentación.
# Tu código a continuación...
asistencia.createOrReplaceTempView("asistencia1")
SesionSpark.sql("select (Nombre_de_Usuario), count(Marca_Temporal) from asistencia1 group by(Nombre_de_Usuario) having count(Marca_Temporal) < 1 ").show()

+-----------------+---------------------+
|Nombre_de_Usuario|count(Marca_Temporal)|
+-----------------+---------------------+
+-----------------+---------------------+



In [23]:
# 6. Consulte los integrantes por cada equipo al que pertenecen.
# Tu código a continuación...
asistencia.createOrReplaceTempView("asistencia1")
SesionSpark.sql("select distinct (Equipo_al_que_perteneces),(Nombre_de_Usuario) from asistencia1 order by 2").show(100)

+------------------------+--------------------+
|Equipo_al_que_perteneces|   Nombre_de_Usuario|
+------------------------+--------------------+
|    Pertenezco a: Ges...| abanolc@unal.edu.co|
|    Pertenezco a: Ges...|acastrillonv@unal...|
|    Pertenezco a: Mi ...| aflemag@unal.edu.co|
|    Pertenezco a: Ges...|anapariciom@unal....|
|    Pertenezco a: Ser...|anapariciom@unal....|
|    Pertenezco a: Adm...|angutierrezb@unal...|
|    Pertenezco a: Te ...|anoriega@unal.edu.co|
|    Pertenezco a: Ges...|auarbelaeza@unal....|
|    Pertenezco a: Ser...| bocampo@unal.edu.co|
|    Pertenezco a: Ges...|cgiraldo@unal.edu.co|
|    Pertenezco a: Ser...|cjfunezg@unal.edu.co|
|    Pertenezco a: Ges...|cquinchiar@unal.e...|
|    Pertenezco a: Mi ...| dadazam@unal.edu.co|
|    Pertenezco a: Lo ...|daestradam@unal.e...|
|    Pertenezco a: Ges...|davgarciava@unal....|
|    Pertenezco a: Te ...|dballesteroso@una...|
|    Pertenezco a: Ges...|  dbrito@unal.edu.co|
|    Pertenezco a: Mi ...|dcadavid@unal.

In [24]:
# 7. Consulte la cantidad de asistentes por presentación, sin considerar los asistentes que pertenecen 
# al equipo que realizó la presentación.
# Tu código a continuación...
asistencia.createOrReplaceTempView("asistencia1")
SesionSpark.sql("""
select count(distinct (Nombre_de_Usuario)), (Equipo_que_va_a_exponer) 
    from asistencia1 
    where (Equipo_al_que_perteneces) != (Equipo_que_va_a_exponer)
group by (Equipo_que_va_a_exponer) 
""").show()

+---------------------------------+-----------------------+
|count(DISTINCT Nombre_de_Usuario)|Equipo_que_va_a_exponer|
+---------------------------------+-----------------------+
|                               80|   Expone: Mi Sanand...|
|                               78|   Expone: Gestión d...|
|                               80|   Expone: Servicios...|
|                               77|   Expone: Gestión d...|
|                               80|   Expone: Administr...|
|                               78|   Expone: Lo tengo ...|
|                               81|   Expone: Gestión d...|
|                               80|   Expone: Te lo cam...|
|                                3|   Expone: Deserción...|
+---------------------------------+-----------------------+



In [25]:
# 8. Consutar cuáles integrantes evaluaron a su propio equipo. Estas evaluaciones no serán válidas, pues un
# integrante no puede evaluar a su propio equipo.
# Tu código a continuación...
evaluacion.createOrReplaceTempView("evaluacion1")
SesionSpark.sql("""
    select distinct e.Nombre_de_Usuario, e.Equipo_que_vas_a_evaluar,e.Equipo_que_vas_a_evaluar
    from evaluacion1 e inner join asistencia1 a
      on a.Nombre_de_Usuario = e.Nombre_de_Usuario
      and replace(a.Equipo_al_que_perteneces,'Pertenezco a: ','') = e.Equipo_que_vas_a_evaluar
""").show()

+--------------------+------------------------+------------------------+
|   Nombre_de_Usuario|Equipo_que_vas_a_evaluar|Equipo_que_vas_a_evaluar|
+--------------------+------------------------+------------------------+
| kmolano@unal.edu.co|    Administración de...|    Administración de...|
|anapariciom@unal....|    Servicios a domic...|    Servicios a domic...|
|hriveraa@unal.edu.co|    Gestión de Menús ...|    Gestión de Menús ...|
|mcarvajalsa@unal....|    Gestión de Reside...|    Gestión de Reside...|
|  lmayar@unal.edu.co|    Gestión de progra...|    Gestión de progra...|
| jruedat@unal.edu.co|    Servicios a domic...|    Servicios a domic...|
|daestradam@unal.e...|    Lo tengo y te lo ...|    Lo tengo y te lo ...|
|jhcordoba@unal.ed...|    Administración de...|    Administración de...|
+--------------------+------------------------+------------------------+



In [26]:
# 9. Consultar la nota promedio por cada ítem (1 al 15), y la nota promedio total del cada equipo. Recuerde que no
# son válidas las evaluaciones realizadas por los miembros del mismo equipo.
# Tu código a continuación...

df = SesionSpark.read.csv("/tmp/Evaluacion_del_Pitch.csv").select("_c0","_c1","_c2","_c3","_c4","_c5","_c6","_c7","_c8","_c9","_c10","_c11","_c12","_c13","_c14","_c15","_c16","_c17","_c18","_c19")
df.createOrReplaceTempView("evaluacionSinObs")

In [27]:
#unicos valores por evaluacion
df2 = SesionSpark.sql("""
    select distinct _c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19
    from evaluacionSinObs e inner join asistencia1 a
      on a.Nombre_de_Usuario = e._c1
      and replace(a.Equipo_al_que_perteneces,'Pertenezco a: ','') <> e._c2
    where e._c3 <> ''
    and e._c3 is not null
    and e._c2 is not null
    and e._c2 <> ''
""")
df2.createOrReplaceTempView("evaluacionesNoEstudianteEvaluandose")

In [28]:
df3 = SesionSpark.sql("""
    select e._c2 Team,
   cast( ( (sum(_c3)/count(_c3) + sum(_c4)/count(_c4) + sum(_c5)/count(_c5) +
    sum(_c6)/count(_c6) + sum(_c7)/count(_c7) + sum(_c8)/count(_c8) +
    sum(_c9)/count(_c9) + sum(_c10)/count(_c10) + sum(_c11)/count(_c11) +
    sum(_c12)/count(_c12) + sum(_c13)/count(_c13) + sum(_c14)/count(_c14) + 
    sum(_c15)/count(_c15) + sum(_c16)/count(_c16) + sum(_c17)/count(_c17)
    ) / 15) as decimal(10,2)) AvgTeam,
    cast(sum(_c3)/count(_c3) as decimal(10,2)) AvgItem1,
    cast(sum(_c4)/count(_c4) as decimal(10,2)) AvgItem2,
    cast(sum(_c5)/count(_c5) as decimal(10,2)) AvgItem3,
    cast(sum(_c6)/count(_c6) as decimal(10,2)) AvgItem4,
    cast(sum(_c7)/count(_c7) as decimal(10,2)) AvgItem5,
    cast(sum(_c8)/count(_c8) as decimal(10,2)) AvgItem6,
    cast(sum(_c9)/count(_c9) as decimal(10,2)) AvgItem7,
    cast(sum(_c10)/count(_c10) as decimal(10,2)) AvgItem8,
    cast(sum(_c11)/count(_c11) as decimal(10,2)) AvgItem9,
    cast(sum(_c12)/count(_c12) as decimal(10,2)) AvgItem10,
    cast(sum(_c13)/count(_c13) as decimal(10,2)) AvgItem11,
    cast(sum(_c14)/count(_c14) as decimal(10,2)) AvgItem12,
    cast(sum(_c15)/count(_c15) as decimal(10,2)) AvgItem13,
    cast(sum(_c16)/count(_c16) as decimal(10,2)) AvgItem14,
    cast(sum(_c17)/count(_c17) as decimal(10,2)) AvgItem15    
    from evaluacionesNoEstudianteEvaluandose e
    group by e._c2 
""");
df3.createOrReplaceTempView("evaluacionesPorGrupo")
df3.show()

+--------------------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+
|                Team|AvgTeam|AvgItem1|AvgItem2|AvgItem3|AvgItem4|AvgItem5|AvgItem6|AvgItem7|AvgItem8|AvgItem9|AvgItem10|AvgItem11|AvgItem12|AvgItem13|AvgItem14|AvgItem15|
+--------------------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+
|Servicios a domic...|   4.42|    4.49|    4.63|    4.52|    4.27|    4.51|    4.61|    4.11|    4.13|    4.35|     4.27|     4.15|     4.65|     4.48|     4.56|     4.59|
|Lo tengo y te lo ...|   4.48|    4.49|    4.60|    4.27|    4.43|    4.43|    4.71|    4.36|    4.26|    4.49|     4.47|     4.36|     4.59|     4.53|     4.60|     4.61|
|Gestión de Reside...|   4.47|    4.55|    4.42|    4.64|    4.45|    4.46|    4.12|    4.52|    4.67|    4.41|     4.52|     4.33|     4.64

In [29]:
# 10. Consulte el mejor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el mejor equipo según el promedio total.
# Tu código a continuación...

from pyspark.sql import functions as F
##
## En este código se ilustra como computar un promedio
## y luego realizar el ordenamiento de la tabla por
## ese mismo promedio.
##
#Max Item1
(df3.select("Team", "AvgItem1") \
    .groupBy("Team") \
    .agg(F.max("AvgItem1").alias("MaxAvgItem1"))) \
        .orderBy("MaxAvgItem1", ascending = False) \
        .show(1) 
#Max Item2
(df3.select("Team", "AvgItem2") \
    .groupBy("Team") \
    .agg(F.max("AvgItem2").alias("MaxAvgItem2"))) \
        .orderBy("MaxAvgItem2", ascending = False) \
        .show(1)
#Max Item3
(df3.select("Team", "AvgItem3") \
    .groupBy("Team") \
    .agg(F.max("AvgItem3").alias("MaxAvgItem3"))) \
        .orderBy("MaxAvgItem3", ascending = False) \
        .show(1)
#Max Item4
(df3.select("Team", "AvgItem4") \
    .groupBy("Team") \
    .agg(F.max("AvgItem4").alias("MaxAvgItem4"))) \
        .orderBy("MaxAvgItem4", ascending = False) \
        .show(1)
#Max Item5
(df3.select("Team", "AvgItem5") \
    .groupBy("Team") \
    .agg(F.max("AvgItem5").alias("MaxAvgItem5"))) \
        .orderBy("MaxAvgItem5", ascending = False) \
        .show(1)
#Max Item6
(df3.select("Team", "AvgItem6") \
    .groupBy("Team") \
    .agg(F.max("AvgItem6").alias("MaxAvgItem6"))) \
        .orderBy("MaxAvgItem6", ascending = False) \
        .show(1)
#Max Item7
(df3.select("Team", "AvgItem7") \
    .groupBy("Team") \
    .agg(F.max("AvgItem7").alias("MaxAvgItem7"))) \
        .orderBy("MaxAvgItem7", ascending = False) \
        .show(1)
#Max Item8
(df3.select("Team", "AvgItem8") \
    .groupBy("Team") \
    .agg(F.max("AvgItem8").alias("MaxAvgItem8"))) \
        .orderBy("MaxAvgItem8", ascending = False) \
        .show(1)
#Max Item9
(df3.select("Team", "AvgItem9") \
    .groupBy("Team") \
    .agg(F.max("AvgItem9").alias("MaxAvgItem9"))) \
        .orderBy("MaxAvgItem9", ascending = False) \
        .show(1)
#Max Item10
(df3.select("Team", "AvgItem10") \
    .groupBy("Team") \
    .agg(F.max("AvgItem10").alias("MaxAvgItem10"))) \
        .orderBy("MaxAvgItem10", ascending = False) \
        .show(1)
#Max Item11
(df3.select("Team", "AvgItem11") \
    .groupBy("Team") \
    .agg(F.max("AvgItem11").alias("MaxAvgItem11"))) \
        .orderBy("MaxAvgItem11", ascending = False) \
        .show(1)
#Max Item12
(df3.select("Team", "AvgItem12") \
    .groupBy("Team") \
    .agg(F.max("AvgItem12").alias("MaxAvgItem12"))) \
        .orderBy("MaxAvgItem12", ascending = False) \
        .show(1)
#Max Item13
(df3.select("Team", "AvgItem13") \
    .groupBy("Team") \
    .agg(F.max("AvgItem13").alias("MaxAvgItem13"))) \
        .orderBy("MaxAvgItem13", ascending = False) \
        .show(1)
#Max Item14
(df3.select("Team", "AvgItem14") \
    .groupBy("Team") \
    .agg(F.max("AvgItem14").alias("MaxAvgItem14"))) \
        .orderBy("MaxAvgItem14", ascending = False) \
        .show(1)
#Max Item15
(df3.select("Team", "AvgItem15") \
    .groupBy("Team") \
    .agg(F.max("AvgItem15").alias("MaxAvgItem15"))) \
        .orderBy("MaxAvgItem15", ascending = False) \
        .show(1)
#Max Team
(df3.select("Team", "AvgTeam") \
    .groupBy("Team") \
    .agg(F.max("AvgTeam").alias("MaxAvgTeam"))) \
        .orderBy("MaxAvgTeam", ascending = False) \
        .show(1) 

+--------------------+-----------+
|                Team|MaxAvgItem1|
+--------------------+-----------+
|Gestión de Reside...|       4.55|
+--------------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|MaxAvgItem2|
+--------------------+-----------+
|Deserción de estu...|       5.00|
+--------------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|MaxAvgItem3|
+--------------------+-----------+
|Deserción de estu...|       5.00|
+--------------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|MaxAvgItem4|
+--------------------+-----------+
|Deserción de estu...|       5.00|
+--------------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|MaxAvgItem5|
+--------------------+-----------+
|Servicios a domic...|       4.51|
+--------------------+-----------+
only showing top 1 row

+----

In [30]:
# 11. Consulte el peor equipo evaluado por cada ítem (según nota promedio. En caso de empate mostrar todos los empatados) 
# y el peor equipo según el promedio total.
# Tu código a continuación...


In [31]:
#min Item1
(df3.select("Team", "AvgItem1") \
    .groupBy("Team") \
    .agg(F.min("AvgItem1").alias("minAvgItem1"))) \
        .orderBy("minAvgItem1", ascending = True) \
        .show(1) 
#min Item2
(df3.select("Team", "AvgItem2") \
    .groupBy("Team") \
    .agg(F.min("AvgItem2").alias("minAvgItem2"))) \
        .orderBy("minAvgItem2", ascending = True) \
        .show(1)
#min Item3
(df3.select("Team", "AvgItem3") \
    .groupBy("Team") \
    .agg(F.min("AvgItem3").alias("minAvgItem3"))) \
        .orderBy("minAvgItem3", ascending = True) \
        .show(1)
#min Item4
(df3.select("Team", "AvgItem4") \
    .groupBy("Team") \
    .agg(F.min("AvgItem4").alias("minAvgItem4"))) \
        .orderBy("minAvgItem4", ascending = True) \
        .show(1)
#min Item5
(df3.select("Team", "AvgItem5") \
    .groupBy("Team") \
    .agg(F.min("AvgItem5").alias("minAvgItem5"))) \
        .orderBy("minAvgItem5", ascending = True) \
        .show(1)
#min Item6
(df3.select("Team", "AvgItem6") \
    .groupBy("Team") \
    .agg(F.min("AvgItem6").alias("minAvgItem6"))) \
        .orderBy("minAvgItem6", ascending = True) \
        .show(1)
#min Item7
(df3.select("Team", "AvgItem7") \
    .groupBy("Team") \
    .agg(F.min("AvgItem7").alias("minAvgItem7"))) \
        .orderBy("minAvgItem7", ascending = True) \
        .show(1)
#min Item8
(df3.select("Team", "AvgItem8") \
    .groupBy("Team") \
    .agg(F.min("AvgItem8").alias("minAvgItem8"))) \
        .orderBy("minAvgItem8", ascending = True) \
        .show(1)
#min Item9
(df3.select("Team", "AvgItem9") \
    .groupBy("Team") \
    .agg(F.min("AvgItem9").alias("minAvgItem9"))) \
        .orderBy("minAvgItem9", ascending = True) \
        .show(1)
#min Item10
(df3.select("Team", "AvgItem10") \
    .groupBy("Team") \
    .agg(F.min("AvgItem10").alias("minAvgItem10"))) \
        .orderBy("minAvgItem10", ascending = True) \
        .show(1)
#min Item11
(df3.select("Team", "AvgItem11") \
    .groupBy("Team") \
    .agg(F.min("AvgItem11").alias("minAvgItem11"))) \
        .orderBy("minAvgItem11", ascending = True) \
        .show(1)
#min Item12
(df3.select("Team", "AvgItem12") \
    .groupBy("Team") \
    .agg(F.min("AvgItem12").alias("minAvgItem12"))) \
        .orderBy("minAvgItem12", ascending = True) \
        .show(1)
#min Item13
(df3.select("Team", "AvgItem13") \
    .groupBy("Team") \
    .agg(F.min("AvgItem13").alias("minAvgItem13"))) \
        .orderBy("minAvgItem13", ascending = True) \
        .show(1)
#min Item14
(df3.select("Team", "AvgItem14") \
    .groupBy("Team") \
    .agg(F.min("AvgItem14").alias("minAvgItem14"))) \
        .orderBy("minAvgItem14", ascending = True) \
        .show(1)
#min Item15
(df3.select("Team", "AvgItem15") \
    .groupBy("Team") \
    .agg(F.min("AvgItem15").alias("minAvgItem15"))) \
        .orderBy("minAvgItem15", ascending = True) \
        .show(1)
#min Team
(df3.select("Team", "AvgTeam") \
    .groupBy("Team") \
    .agg(F.min("AvgTeam").alias("minAvgTeam"))) \
        .orderBy("minAvgTeam", ascending = True) \
        .show(1) 

+--------------------+-----------+
|                Team|minAvgItem1|
+--------------------+-----------+
|Administración de...|       4.13|
+--------------------+-----------+
only showing top 1 row

+---------------+-----------+
|           Team|minAvgItem2|
+---------------+-----------+
|Mi Sanandresito|       3.90|
+---------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|minAvgItem3|
+--------------------+-----------+
|Administración de...|       3.97|
+--------------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|minAvgItem4|
+--------------------+-----------+
|Gestión de progra...|       3.90|
+--------------------+-----------+
only showing top 1 row

+--------------------+-----------+
|                Team|minAvgItem5|
+--------------------+-----------+
|Administración de...|       4.19|
+--------------------+-----------+
only showing top 1 row

+--------------------+--------

In [32]:
# 12. Consulte la lista de estudiantes con la correspondiente nota obtenida en la presentación 
# (nota promedio total de la evaluación obtenida por el equipo)
# Tu código a continuación...

In [34]:
#Estudiantes y grupo pertenecen
df4 = SesionSpark.sql("""
    select a.Nombre_de_Usuario,e.Team, AvgTeam
    from evaluacionesPorGrupo e inner join asistencia1 a
      on replace(a.Equipo_al_que_perteneces,'Pertenezco a: ','') = e.Team
    group by a.Nombre_de_Usuario,e.Team,AvgTeam
      order by a.Nombre_de_Usuario
""")
df4.show(100)

+--------------------+--------------------+-------+
|   Nombre_de_Usuario|                Team|AvgTeam|
+--------------------+--------------------+-------+
| abanolc@unal.edu.co|Gestión de Menús ...|   4.36|
|acastrillonv@unal...|Gestión de progra...|   4.39|
| aflemag@unal.edu.co|     Mi Sanandresito|   4.20|
|anapariciom@unal....|Servicios a domic...|   4.42|
|anapariciom@unal....|Gestión de Menús ...|   4.36|
|angutierrezb@unal...|Administración de...|   4.19|
|anoriega@unal.edu.co|       Te lo cambio.|   4.29|
|auarbelaeza@unal....|Gestión de Reside...|   4.47|
| bocampo@unal.edu.co|Servicios a domic...|   4.42|
|cgiraldo@unal.edu.co|Gestión de Reside...|   4.47|
|cjfunezg@unal.edu.co|Servicios a domic...|   4.42|
|cquinchiar@unal.e...|Gestión de Reside...|   4.47|
| dadazam@unal.edu.co|     Mi Sanandresito|   4.20|
|daestradam@unal.e...|Lo tengo y te lo ...|   4.48|
|davgarciava@unal....|Gestión de Menús ...|   4.36|
|dballesteroso@una...|       Te lo cambio.|   4.29|
|  dbrito@un